In [1]:
from azure.quantum import Workspace
workspace = Workspace(
            resource_id = "/subscriptions/2f8b131d-6276-4fe0-be2d-4646cd04042a/resourceGroups/classicalcomputing/providers/Microsoft.Quantum/Workspaces/classicalcomputing",
            location = "eastus")

In [2]:
import qsharp

Preparing Q# environment...


In [3]:
%%qsharp

open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation QFTImpl (qs : Qubit[]) : Unit is Adj + Ctl
{
    body (...)
    {
        let nQubits = Length(qs);

        for i in 0 .. nQubits - 1
        {
            H(qs[i]);
            for j in i + 1 .. nQubits - 1
            {
                Controlled R1Frac([qs[j]], (1, j - i, qs[i]));
            }
        }

        Microsoft.Quantum.Canon.SwapReverseRegister(qs);
    }
}


In [4]:
%%qsharp

operation QuantumAdd (x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(x);
    QFTImpl(y);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            Controlled R1Frac([x[i + j]], (2, j + 1, (y)[(n - 1) - i]));
        }
    }
    Adjoint QFTImpl(y);
}

operation QuantumAddByNumber (x : Qubit[], b : Int) : Unit is Adj + Ctl {
    let n = Length(x);

    // apply Draper adder for numeric
    QFTImpl(x);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            if(not((b / 2^((n - 1) - (i + j))) % 2 == 0)) {
                R1Frac(2, j + 1, (x)[(n - 1) - i]);
            }
        }
    }
    Adjoint QFTImpl(x);
}

operation QuantumAddByModulus (N : Int, x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    use (ancilla, cx, cy) = (Qubit(), Qubit(), Qubit()) {
        // add bit for preventing overflow
        let x_large = [cx] + x;
        let y_large = [cy] + y;
        // |x⟩ |y⟩ -> |x⟩ |x + y⟩
        QuantumAdd(x_large, y_large);
        // |y⟩ -> |y - N⟩
        Adjoint QuantumAddByNumber(y_large, N);
        // Turn on ancilla when first bit is |1⟩ (i.e, when x + y - N < 0)
        Controlled X([y_large[0]], ancilla);
        // Add N back when ancilla is |1⟩
        Controlled QuantumAddByNumber([ancilla], (y_large, N));
        // set ancilla to |0⟩
        Adjoint QuantumAdd(x_large, y_large);
        X(ancilla);
        Controlled X([y_large[0]], ancilla);
        QuantumAdd(x_large, y_large);
    }
}

operation QuantumMultiplyByModulus (N : Int, a : Int, y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(y);
    let a_mod = a % N;

    use s = Qubit[n] {
        // start |y⟩ |0⟩

        // apply adder by repeating "a" (integer) times
        for r in 0 .. a_mod - 1 {
            QuantumAddByModulus(N, y, s);
        }
        // now |y⟩ |a y mod N⟩

        // swap first register and second one by tuple
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(y, s));
        // now |a y mod N⟩ |y⟩

        // reset all s qubits !
        // but it's tricky because we cannot use "Reset()" since here is controlled operator.
        let a_inv = InverseModI(a_mod, N);
        for r in 0 .. a_inv - 1 {
            Adjoint QuantumAddByModulus(N, y, s);
        }
    }
}


In [5]:
%%qsharp

operation QuantumExponentForPeriodFinding (a : Int, N : Int, x : Qubit[], y : Qubit[]) : Unit {
    let n1 = Length(x);
    let n2 = Length(y);

    // set |y⟩ = |0...01⟩
    X(y[n2 - 1]);

    for idx in 0 .. n1 - 1 {
        // a^(2^((n1-1) - idx)) is too big, then we reduce beforehand
        mutable a_mod = 1;
        for power in 1 .. 2^((n1-1) - idx) {
            set a_mod = (a_mod * a) % N;
        }
        // apply decomposition elements
        Controlled QuantumMultiplyByModulus([x[idx]], (N, a_mod, y));
    }
}


In [6]:
%%qsharp

operation QuantumPeriodFinding(num : Int, a : Int) : Int {
    let n1 = BitSizeI(num) * 2;
    let n2 = BitSizeI(num);
    mutable periodCandidate = 1;
    repeat {
        use (x, y) = (Qubit[n1], Qubit[n2]) {
            Microsoft.Quantum.Canon.ApplyToEachCA(H, x);

            QuantumExponentForPeriodFinding(a, num, x, y);

            mutable tmpResult = new Result[n2];
            for idx in 0 .. n2 - 1 {
                set tmpResult w/= idx <-MResetZ(y[idx]);
            }

            QFTImpl(x);

            mutable realResult = new Result[n1];
            for idx in 0 .. n1 - 1 {
                set realResult w/= idx <-MResetZ(x[idx]);
            }
            
            let resultBool = [false] + Microsoft.Quantum.Convert.ResultArrayAsBoolArray(realResult); 
            let resultBool_R = Microsoft.Quantum.Arrays.Reversed(resultBool);
            let resultIntL = Microsoft.Quantum.Convert.BoolArrayAsBigInt(resultBool_R);

            let gcdL = GreatestCommonDivisorL(resultIntL, 2L^n1);
            let calculatedNumerator = resultIntL / gcdL;
            let calculatedDenominator = 2L^n1 / gcdL;
            let numL = Microsoft.Quantum.Convert.IntAsBigInt(num);
            let approximatedFraction =
                ContinuedFractionConvergentL(BigFraction(calculatedNumerator, calculatedDenominator), numL);
            let (approximatedNumerator, approximatedDenominator) = approximatedFraction!;
            mutable periodCandidateL = 0L;
            if(approximatedDenominator < 0L) {
                set periodCandidateL = approximatedDenominator * -1L;
            }
            else {
                set periodCandidateL = approximatedDenominator;             
            }
            set periodCandidate = ReduceBigIntToInt(periodCandidateL);

            Message($"Fração : {resultIntL} / {2L^n1}");
            Message($"Fração aproximada : {approximatedNumerator} / {approximatedDenominator}");
            Message($"Periodo candidato : {periodCandidate}");
        }
    }
    until ((periodCandidate != 0) and (ExpModI(a, periodCandidate, num) == 1))
    fixup {
    }

    return periodCandidate;
}

operation ReduceBigIntToInt(numL : BigInt) : Int {
    
    Microsoft.Quantum.Diagnostics.Fact(BitSizeL(numL) <= 32, $"Cannot convert to Int. Input is too large");

    mutable resultInt = 0;
    let numArray = Microsoft.Quantum.Convert.BigIntAsBoolArray(numL);
    let numArray_R = Microsoft.Quantum.Arrays.Reversed(numArray);
    let nSize = Length(numArray_R);
    for idx in 0 .. nSize - 1 {
        if(numArray_R[idx] and ((nSize - 1) - idx <= 31)) {
            set resultInt = resultInt + (2 ^ ((nSize - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(15,33): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(24,34): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [7]:
from math import gcd

# Defina os valores de N e a
N_values = [15]
a_values = [7]
period_candidate = 4

print(tuple(zip(N_values, a_values)))

# Simula a busca de período quântico para cada par de N e a
for N, a in zip(N_values, a_values):
    # Agora você pode usar period_candidate automaticamente para cada par de N e a
    print(f"Para N = {N} e a = {a}")

    period_candidate = QuantumPeriodFinding.simulate(num=N, a=a)


print(f"Saindo da busca de período com period_candidate = {period_candidate}")


((15, 7),)
Para N = 15 e a = 7
Fração : 192 / 256
Fração aproximada : 3 / 4
Periodo candidato : 4
Saindo da busca de período com period_candidate = 4


In [20]:
import qsharp
import time
import math
from sympy import randprime

a = 5 

n = randprime(1000000000, 10000000000) 
print(f"Número primo gerado: {n}")
print("-" * 40)

periodCandidate = period_candidate
print("Periodo candidato encontrado: ",period_candidate)

start_time = time.time()

if periodCandidate % 2 == 0 and pow(a, periodCandidate // 2, n) != n - 1:
    factor1 = math.gcd(pow(a, periodCandidate // 2, n) - 1, n)
    factor2 = n // factor1
    print(f"Fatores encontrados: {factor1}, {factor2}")
else:
    print("Período não válido para fatorização.")

end_time = time.time()
execution_time = end_time - start_time

print(f"Tempo de execução: {execution_time:.6f} segundos")
print("-" * 40)

Número primo gerado: 9700791967
----------------------------------------
Periodo candidato encontrado:  4
Fatores encontrados: 1, 9700791967
Tempo de execução: 0.000168 segundos
----------------------------------------


In [24]:
import qsharp
import time
import math
from sympy import randprime

def gerar_numero_composto(valor_minimo, valor_maximo):
    while True:
        num = randprime(valor_minimo, valor_maximo)
        if num > 3: 
            return num

valor_minimo = 1000000000
valor_maximo = 10000000000
num_testes = 1

for _ in range(num_testes):
    n = gerar_numero_composto(valor_minimo, valor_maximo)
    print("-" * 40)
    print(f"Número composto gerado: {n}")

    start_time = time.time()

    a = 5 

    periodCandidate = period_candidate
    print("Periodo candidato encontrado: ",period_candidate)

    if period_candidate is None:
        print("Período inválido para fatorização.")
    else:
        fator1 = math.gcd(pow(a, period_candidate // 2, n) - 1, n)
        fator2 = n // fator1
        print(f"Fatores encontrados: {fator1}, {fator2}")

    end_time = time.time()
    tempo_execucao = end_time - start_time

    print(f"Tempo de execução: {tempo_execucao:.6f} segundos")
    print("-" * 40)


----------------------------------------
Número composto gerado: 7814596608677
Periodo candidato encontrado:  4
Fatores encontrados: 1, 7814596608677
Tempo de execução: 0.000040 segundos
----------------------------------------
